---
title: "2023 Survey"
format:
  html:
    toc: true
    css: styles.css
    echo: true
    code-fold: true
---


In [ ]:
#| echo: false
%reload_ext autoreload
%autoreload 2

from pathlib import Path

import numpy as np
import pandas as pd
import toml
from IPython.display import HTML
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "notebook"

CONFIG  = toml.load('config.toml')

In [ ]:
#| code-summary: "Data"
xlsx_path = (
    Path(CONFIG['obs_2023']['paths']['obs_dir'])
    /CONFIG['obs_2023']['paths']['od_xlsx']
)
results_sheet = CONFIG['obs_2023']['sheets']['results']

obs = (
    pd.read_excel(xlsx_path, sheet_name=results_sheet)
    .set_index('ID')
)

## Demographics

### Age

In [ ]:
#| code-summary: "Data"
# Categorize based on HHTS categories
hhts_ages = {
    'Under 5': 5,
    '5-15': 15,
    '16-17': 17,
    '17-18': 18,
    '25-34': 34,
    '35-44': 44,
    '45-54': 54,
    '55-64': 64,
    '65-74': 74,
    '75-84': 84,
    '85 or Older': np.inf,
}

age = pd.DataFrame(
    {
        'age': (obs.DATE_COMPLETED.dt.year - obs.YEAR_BORN),
        'age_category': pd.cut(
            # Calculate age based on survey date and respondents birth year
            x=(obs.DATE_COMPLETED.dt.year - obs.YEAR_BORN),
            bins=[0] + list(hhts_ages.values()),
            labels=list(hhts_ages.keys())
        ),
        'unlinked_weight': obs.UNLINKED_WGHT_FCTR,
        'linked_weight': obs.LINKED_WGHT_FCTR,
    }
)

In [ ]:
#| code-summary: "Table"

unweighted = age.groupby('age_category', observed=True).size()
unlinked = age.groupby('age_category', observed=True).unlinked_weight.sum()
linked = age.groupby('age_category', observed=True).linked_weight.sum()
total_valid = age.dropna().size
total = age.size

summary = pd.DataFrame(
    {
        ('unweighted', 'count'): unweighted,
        ('unweighted', 'percent'): unweighted / unweighted.sum(),
        ('unlinked_weight', 'count'): unlinked,
        ('unlinked_weight', 'percent'): unlinked / unlinked.sum(),
        ('linked_weight', 'count'): linked,
        ('linked_weight', 'percent'): linked / linked.sum(),
    }
)

table = (
    pd.concat(
        [
            summary,
            pd.DataFrame(
                [
                    {'age_category': 'Total valid', ('unweighted', 'count'): total_valid, ('unweighted', 'percent'): total_valid/total},
                    {'age_category': 'Total', ('unweighted', 'count'): total}
                ],
                index=['Total valid', 'Total'],
            )
            .set_index('age_category')
            
        ]
    )
    .to_html(
        formatters={
            ('unweighted', 'count'): '{:,.0f}'.format,
            ('unweighted', 'percent'): '{:,.2%}'.format,
            ('unlinked_weight', 'count'): '{:,.0f}'.format,
            ('unlinked_weight', 'percent'): '{:,.2%}'.format,
            ('linked_weight', 'count'): '{:,.0f}'.format,
            ('linked_weight', 'percent'): '{:,.2%}'.format,
        }
    )
)

display(HTML(table))

In [ ]:
#| code-summary: "Chart"

figure = go.Figure()
figure.add_trace(
    go.Bar(
        x=summary.index,
        y=summary[('unweighted', 'percent')],
        name='Unweighted',
    )
)
figure.add_trace(
    go.Bar(
        x=summary.index,
        y=summary[('unlinked_weight', 'percent')],
        name='Weighted (Unlinked)',
    )
)
figure.add_trace(
    go.Bar(
        x=summary.index,
        y=summary[('linked_weight', 'percent')],
        name='Weighted (Linked)',
    )
)
figure.update_layout(
    title_text='Unweighted and Weighted Responses by Age',
    yaxis_range=[0, 1],
    yaxis_tickformat=',.0%',
    xaxis_title='Age',
yaxis_title='Percent',
)

figure.show()

### Income